# Modkit dmr
# Day 6 data
## Use my Filtered Reads

In [ ]:
from datetime import datetime

def current_time():
    """Returns the current date and time as a formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S") 

print("Current Date and Time:", current_time())

Based on:
https://nanoporetech.github.io/modkit/intro_dmr.html#perform-differential-methylation-scoring

Select kernal: dimelo_v2_modkit_parsing

Preparing the input data
The inputs to all modkit dmr commands are two or more bedMethyl files (created by modkit pileup) that have been compressed with bgzip and indexed with tabix. An example of how to generate the input data is shown below:


ref=grch38.fasta
threads=32

norm=normal_sample.bam
norm_pileup=normal_pileup.bed

modkit pileup ${norm} ${norm_pileup} \
  --cpg \
  --ref ${ref} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${norm_pileup}
tabix -p bed ${norm_pileup}.gz

# pileup and compression can also be done in one step
tumor=tumor_sample.bam
tumor_pileup=tumor_pileup.bed.gz

modkit pileup ${tumor} - \
  --cpg \
  --ref ${ref} \
  --threads ${threads} \
  --log-filepath log.txt | ${bgzip} -c > ${tumor_pileup}

tabix -p bed ${tumor_pileup}

In [ ]:
%%bash
echo "hello"

# Use the NEW modkit latest installed version in ipython kernel modkit_new

In [ ]:
# ! python3 -m ipykernel install --user --name=modkit_new --display-name "modkit_new Python"
# ! which modkit

In [ ]:
import os
os.environ["PATH"] = "/home/michalula/.cargo/bin:" + os.environ["PATH"]
! which modkit
! modkit --version

In [ ]:
# ! modkit

In [ ]:
! modkit --version 

In [ ]:
import os
import pandas as pd

def load_pileup_bed(bed_path):
    # bed_path = existing[0]
    print("Reading bedMethyl file:", bed_path)

    # bedMethyl column names (18 columns as provided)
    colnames = [
        "chrom", "start", "end", "mod_code", "score", "strand",
        "start2", "end2", "color",
        "Nvalid_cov", "percent_modified", "Nmod", "Ncanonical",
        "Nother_mod", "Ndelete", "Nfail", "Ndiff", "Nnocall"
    ]

    # Configure dtypes where reasonable
    dtypes = {
        "chrom": str,
        "start": "Int64",
        "end": "Int64",
        "mod_code": str,
        "score": "Int64",
        "strand": str,
        "start2": "Int64",
        "end2": "Int64",
        "color": str,
        "Nvalid_cov": "Int64",
        "percent_modified": float,
        "Nmod": "Int64",
        "Ncanonical": "Int64",
        "Nother_mod": "Int64",
        "Ndelete": "Int64",
        "Nfail": "Int64",
        "Ndiff": "Int64",
        "Nnocall": "Int64"
    }

    compression = "gzip" if bed_path.endswith(".gz") else None

    # Read file (headerless BED-like table). If file has extra columns, keep them with automatic numeric conversion below.
    df = pd.read_csv(
        bed_path,
        sep="\t",
        header=None,
        comment="#",
        names=colnames,
        dtype=dtypes,
        compression=compression,
        engine="python",
        na_values=[".", "NA", ""],
        keep_default_na=True
    )

    # If file contained more than 18 columns, pandas assigned remaining data to extra columns named like col_18, col_19...
    # Ensure numeric conversion for numeric-like columns
    for c in df.columns:
        if df[c].dtype == object:
            # try safe numeric conversion where appropriate
            try:
                df[c] = pd.to_numeric(df[c], errors="ignore")
            except Exception:
                pass

    print("Loaded DataFrame shape:", df.shape)
    display(df.head())
    return df


In [ ]:
import os
from IPython.display import display, HTML
from plotly import express as px
from plotly import graph_objects as go

# ! python3 -m pip install plotly
# ! python3 -m pip install matplotlib
# ! python3 -m pip install nbformat>=4.2.0

def plot_pileup_roi_df(df_roi, out_dir):
    os.makedirs(out_dir, exist_ok=True)
    # ensure numeric types for plotting
    df_roi['pos'] = df_roi['start'].astype(int)
    df_roi['percent_modified'] = df_roi['percent_modified'].astype(float)
    df_roi['Nvalid_cov'] = df_roi['Nvalid_cov'].astype(int)
    df_roi['Nmod'] = df_roi['Nmod'].astype(int)
    df_roi['Ncanonical'] = df_roi['Ncanonical'].astype(int)

    # Scatter: genomic position vs percent modified (point size = coverage)
    fig1 = px.scatter(
        df_roi,
        x='pos',
        y='percent_modified',
        color='strand',
        size='Nvalid_cov',
        hover_data=['Nvalid_cov','Nmod','Ncanonical','Nother_mod','Nnocall'],
        title='Percent modified across ROI (size = Nvalid_cov)',
        height=500
    )
    fig1.update_layout(xaxis_title='Genomic position (start)', yaxis_title='Percent modified')
    fig1.show()
    # fig1.write_html(os.path.join(out_dir, "roi_percent_modified_scatter.html"), include_plotlyjs='cdn')

    # Histogram: coverage distribution
    fig2 = px.histogram(
        df_roi,
        x='Nvalid_cov',
        nbins=40,
        title='Distribution of Nvalid_cov (coverage) in ROI',
        height=400
    )
    fig2.update_layout(xaxis_title='Nvalid_cov', yaxis_title='Count')
    fig2.show()
    # fig2.write_html(os.path.join(out_dir, "roi_nvalidcov_hist.html"), include_plotlyjs='cdn')

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked)
    topn = 30
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'Sorted Top {topn} sites by percent_modified (stacked Nmod / Ncanonical)',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Print simple summaries
    print("ROI rows:", df_roi.shape[0])
    print("Percent modified: median={:.2f}, mean={:.2f}".format(df_roi['percent_modified'].median(), df_roi['percent_modified'].mean()))
    print("Coverage (Nvalid_cov): min={}, median={}, max={}".format(df_roi['Nvalid_cov'].min(), df_roi['Nvalid_cov'].median(), df_roi['Nvalid_cov'].max()))

    # Display first rows table for quick inspection
    display(HTML(df_roi.head(20).to_html(index=False)))


    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked)
    topn = df_roi.shape[0]
    # df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_roi.copy() #.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'All {topn} CpG sites by percent_modified (stacked Nmod / Ncanonical) [ordered=not s]',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked, NOT SORTED)
    topn = df_roi.shape[0]
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    fig3 = go.Figure()
    fig3.add_trace(go.Bar(name='Nmod', x=df_top['label'], y=df_top['Nmod']))
    fig3.add_trace(go.Bar(name='Ncanonical', x=df_top['label'], y=df_top['Ncanonical']))
    fig3.update_layout(barmode='stack', title=f'Sorted Top {topn} sites by percent_modified (stacked Nmod / Ncanonical)',
                    xaxis_title='position:strand', yaxis_title='reads', height=520)
    fig3.show()
    # fig3.write_html(os.path.join(out_dir, "roi_top_sites_stacked_counts.html"), include_plotlyjs='cdn')

    # Print simple summaries
    print("ROI rows:", df_roi.shape[0])
    print("Percent modified: median={:.2f}, mean={:.2f}".format(df_roi['percent_modified'].median(), df_roi['percent_modified'].mean()))
    print("Coverage (Nvalid_cov): min={}, median={}, max={}".format(df_roi['Nvalid_cov'].min(), df_roi['Nvalid_cov'].median(), df_roi['Nvalid_cov'].max()))

    # Display first rows table for quick inspection
    display(HTML(df_roi.head(20).to_html(index=False)))

    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    topn = 30
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig4.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig4.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig4.show()
    # fig4.write_html(os.path.join(out_dir, "roi_top_sites_stacked_percentage.html"), include_plotlyjs='cdn')     


    # Bar: top sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    topn = 277
    df_top = df_roi.sort_values('percent_modified', ascending=False).head(topn).copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig4 = go.Figure()
    fig4.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig4.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig4.update_layout(barmode='stack', title=f'Top {topn} sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig4.show()
    # fig4.write_html(os.path.join(out_dir, "roi_top_sites_stacked_percentage.html"), include_plotlyjs='cdn')     

    # Bar: Unsorted sites by percent_modified showing Nmod vs Ncanonical (stacked) percentages
    df_top = df_roi.copy()
    df_top = df_top.assign(label=df_top['pos'].astype(str) + ":" + df_top['strand'])
    df_top = df_top.assign(Ntotal=df_top['Nmod'] + df_top['Ncanonical'])
    df_top = df_top.assign(Nmod_perc=(df_top['Nmod'] / df_top['Ntotal']) * 100)
    df_top = df_top.assign(Ncanonical_perc=(df_top['Ncanonical'] / df_top['Ntotal']) * 100)
    fig5 = go.Figure()
    fig5.add_trace(go.Bar(name='Nmod %', x=df_top['label'], y=df_top['Nmod_perc']))
    fig5.add_trace(go.Bar(name='Ncanonical %', x=df_top['label'], y=df_top['Ncanonical_perc']))
    fig5.update_layout(barmode='stack', title=f'All sites by percent_modified (stacked Nmod % / Ncanonical %)',
                    xaxis_title='position:strand', yaxis_title='percentage', height=520)
    fig5.show()
    # fig5.write_html(os.path.join(out_dir, "roi_all_sites_stacked_percentage.html"), include_plotlyjs='cdn')    


    return df_top



In [ ]:
! ls /home/michalula/data/ref_genomes/t2t_v2_0/

# Pileups 
## for CRISPRoff filtered data for Day 6

In [ ]:
! ls /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/CG_137_padded_reads_day6_CRoff_Tcells_mC0.7_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10_mCthresh0.7_t2t_v2_0_chr1:206583354-206589854_2025-09-29_units_combined_numFWD802_numRVS1480.npy

In [ ]:
! ls /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/

In [ ]:
%%bash
date_today="20251110"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_pileup/"
mkdir ${pileup_data_folder_path}

# mC > 99.5% = 0995 filtered 
# filtered_CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# mC >70% = 07 filtered 
# filtered_CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# pileup_CROFF_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

# Pre-filtered mC > 70%::
CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_day6_CRoff_Tcells_mC0.7_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10.bam"
pileup_CROFF_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"


# use full data unfiltered .BAM file
# data_folder_path="/home/michalula/data/cas9_nanopore/data/20250908_nCATs_T_CRoff_Day_6/5mCG/to_t2t_v2_0/"
# CROFF_day6_bam=${data_folder_path}"sort_align_t2t_v2_0_trim_20250908_Day6_CROFF_Tcells_2Libraries_Minion_R9.dna_r9.4.1_e8_sup@v3.3.5mCG.bam"
# pileup_CROFF_day6_bed=${data_folder_path}${date_today}"_full_data""_pileup_CROFF_Day6_Tcells.bed"


ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref= "/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"
threads=32


modkit pileup ${CROFF_day6_bam} ${pileup_CROFF_day6_bed} \
  --cpg \
  --ref ${ref_genome_fa} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${pileup_CROFF_day6_bed}
tabix -p bed ${pileup_CROFF_day6_bed}.gz

printf '%s\n' "CROFF_day6_bam: $CROFF_day6_bam"
printf '%s\n' "pileup_CROFF_day6_bed: $pileup_CROFF_day6_bed"
cat "$pileup_CROFF_day6_bed"

## Pileup columns explore

bedMethyl column descriptions.

Definitions:

Nmod - Number of calls passing filters that were classified as a residue with a specified base modification.

Ncanonical - Number of calls passing filters were classified as the canonical base rather than modified. The exact base must be inferred by the modification code. For example, if the modification code is m (5mC) then the canonical base is cytosine. If the modification code is a, the canonical base is adenine.

Nother mod - Number of calls passing filters that were classified as modified, but where the modification is different from the listed base (and the corresponding canonical base is equal). For example, for a given cytosine there may be 3 reads with h calls, 1 with a canonical call, and 2 with m calls. In the bedMethyl row for h Nother_mod would be 2. In the m row Nother_mod would be 3.

Nvalid_cov - the valid coverage. Nvalid_cov = Nmod + Nother_mod + Ncanonical, also used as the score in the bedMethyl

Ndiff - Number of reads with a base other than the canonical base for this modification. For example, in a row for h the canonical base is cytosine, if there are 2 reads with C->A substitutions, Ndiff will be 2.

Ndelete - Number of reads with a deletion at this reference position

Nfail - Number of calls where the probability of the call was below the threshold. The threshold can be set on the command line or computed from the data (usually failing the lowest 10th percentile of calls).

Nnocall - Number of reads aligned to this reference position, with the correct canonical base, but without a base modification call. This can happen, for example, if the model requires a CpG dinucleotide and the read has a CG->CH substitution such that no modification call was produced by the basecaller.


column	name	description	type

    1	chrom	name of reference sequence from BAM header	str

    2	start position	0-based start position	int

    3	end position	0-based exclusive end position	int

    4	modified base code and motif	single letter code for modified base and motif when more than one motif is used	str

    5	score	equal to Nvalid_cov	int

    6	strand	'+' for positive strand '-' for negative strand, '.' when strands are combined	str

    7	start position	included for compatibility	int

    8	end position	included for compatibility	int

    9	color	included for compatibility, always 255,0,0	str

    10	Nvalid_cov	see definitions above.	int

    11	percent modified	(Nmod / Nvalid_cov) * 100	float

    12	Nmod	see definitions above	int

    13	Ncanonical	see definitions above	int

    14	Nother_mod	see definitions above	int

    15	Ndelete	see definitions above	int

    16	Nfail	see definitions above	int

    17	Ndiff	see definitions above	int

    18	Nnocall	see definitions above	int


In [ ]:

date_today="20251110"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=data_folder_path+"new_pileup/"

CROFF_day6_bam=data_folder_path+"filtered_reads_overlap_MORE_than_0.9_day6_CRoff_Tcells_mC0.7_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10.bam"
pileup_CROFF_day6_bed=pileup_data_folder_path+date_today+"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

pileup_CROFF_day6_bed

In [ ]:
# date_today="20251110"

# data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/replica_1/analyze_single_reads/dimelo_v2_output/"
# pileup_data_folder_path=data_folder_path+"new_pileup/"
# # mkdir ${pileup_data_folder_path}

# # mkdir ${pileup_data_folder_path}
# # mC > 99.5% = 0995 filtered 
# # filtered_CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# # mC >70% = 07 filtered 
# # filtered_CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# # pileup_CROFF_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

# # mC > 99.5% = 0995 filtered 
# # filtered_CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# # filtered_CROFF_day6_bam=data_folder_path+"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# # mC >70% = 07 filtered 
# filtered_CROFF_day6_bam=data_folder_path+"filtered_reads_overlap_MORE_than_0.9_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.995_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# pileup_CROFF_day6_bed=pileup_data_folder_path+date_today+"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

# # Pre-filtered mC > 70%::
# # CROFF_day6_bam=${data_folder_path}"pre_filtered_ROI_reads_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# # pileup_CROFF_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

# # Pre-filtered mC > 70%::pileup_CROFF_day6_bed
# CROFF_day6_bam=data_folder_path+"pre_filtered_ROI_reads_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# pileup_CROFF_day6_bed=pileup_data_folder_path+date_today+"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

# pileup_CROFF_day6_bed


In [ ]:
pileup_CROFF_day6_df = load_pileup_bed(pileup_CROFF_day6_bed)
pileup_CROFF_day6_df

In [ ]:
pileup_CROFF_day6_df_stats = plot_pileup_roi_df(df_roi=pileup_CROFF_day6_df, out_dir=pileup_data_folder_path)
pileup_CROFF_day6_df_stats


#TODO:
# WHAT is going on with the lack of C calls at the target dip????

# <!> WHY is the total number of mC+C reads SO LOW????

# WHY is the last figure with precentages SO MUCH cleaner and hides the LACK of called based info there????

in that C called as something else???

Is that not a real ROI Cas9 dip??

WHAT Are the bases there?????

AAA those are SORTED???? !!!!

based on which column????

# Look at CpGs within out target ROI
T2T v2.0

First CG:
206583388,206583390

Last of selected 137 CGs in the ROI:

206589746,206589748 --CpG_137

=> here have each CG position separate: so have 137*2  = 276

In [ ]:
137*2, 277-5

In [ ]:
pileup_CROFF_day6_df[pileup_CROFF_day6_df['start'] == 206583387]

In [ ]:
pileup_CROFF_day6_df[pileup_CROFF_day6_df['start'] == 206583388]

In [ ]:
pileup_CROFF_day6_df[pileup_CROFF_day6_df['start'] == 206589746]

In [ ]:
(284-10) / 2

In [ ]:
pileup_CROFF_day6_df_roi = pileup_CROFF_day6_df.iloc[10:284, :]  # Display target region rows
print(pileup_CROFF_day6_df_roi.shape,pileup_CROFF_day6_df_roi.shape[0]/2)
pileup_CROFF_day6_df_roi

<!>
From day 6 data:
Threshold of  0.767

From day 35 data:
> Threshold of  0.7597656 for base C is low. Consider increasing the filter-percentile or specifying a higher threshold.
> Done, processed 11762972 rows. Processed ~129977 reads and skipped ~150 reads.

In [ ]:
# Plot pileup_Unedit_day6_df_roi summary plots (use existing variables/imports in the notebook)
# Saves interactive HTMLs to pileup_data_folder_path and displays inline.
out_dir = pileup_data_folder_path  # existing variable in the notebook

df_roi = pileup_CROFF_day6_df_roi.copy()

df_roi_stats = plot_pileup_roi_df(df_roi=pileup_CROFF_day6_df_roi, out_dir=pileup_data_folder_path)
df_roi_stats


# Unedited T cells Day 6

In [ ]:
! ls /home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/unedited/analyze_single_reads/dimelo_v2_output

In [ ]:
%%bash

date_today="20251110"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/unedited/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_pileup/"
mkdir ${pileup_data_folder_path}

filtered_Unedit_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_day6_unedited_Tcells_mC0.995_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10.bam"
pileup_Unedit_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_NT_Day6_Tcells.bed"
cat "$pileup_Unedit_day6_bed"

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref="/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"
threads=32

modkit pileup ${filtered_Unedit_day6_bam} ${pileup_Unedit_day6_bed} \
  --cpg \
  --ref ${ref_genome_fa} \
  --threads ${threads} \
  --log-filepath log.txt

bgzip -k ${pileup_Unedit_day6_bed}
tabix -p bed ${pileup_Unedit_day6_bed}.gz

printf '%s\n' "filtered_Unedit_day6_bam: $filtered_Unedit_day6_bam"
printf '%s\n' "pileup_Unedit_day6_bed: $pileup_Unedit_day6_bed"
cat "$pileup_Unedit_day6_bed"


In [ ]:
date_today="20251110"
data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/unedited/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=data_folder_path+"new_pileup/"
pileup_Unedit_day6_bed=pileup_data_folder_path+date_today+"_filtered_mC07_pileup_NT_Day6_Tcells.bed"
# cat "$pileup_Unedit_day6_bed"


pileup_Unedit_day6_bed

In [ ]:
pileup_Unedit_day6_df = load_pileup_bed(pileup_Unedit_day6_bed)
pileup_Unedit_day6_df

In [ ]:
pileup_Unedit_day6_df_stats = plot_pileup_roi_df(df_roi=pileup_Unedit_day6_df, out_dir=pileup_data_folder_path)
pileup_Unedit_day6_df_stats

# focus on ROI

In [ ]:
pileup_Unedit_day6_df[pileup_Unedit_day6_df['start'] == 206583388-1]

In [ ]:
pileup_Unedit_day6_df[pileup_Unedit_day6_df['start'] == 206583388]

In [ ]:
pileup_Unedit_day6_df[pileup_Unedit_day6_df['start'] == 206589746-1]

In [ ]:
pileup_Unedit_day6_df[pileup_Unedit_day6_df['start'] == 206589746]

In [ ]:
pileup_Unedit_day6_df_roi = pileup_Unedit_day6_df.iloc[4:278, :]  # Display target region rows
print(pileup_Unedit_day6_df_roi.shape,pileup_Unedit_day6_df_roi.shape[0]/2)
pileup_Unedit_day6_df_roi

In [ ]:
pileup_Unedit_day6_df

In [ ]:
pileup_Unedit_day6_df_roi

In [ ]:
pileup_data_folder_path

In [ ]:
df_roi_stats = plot_pileup_roi_df(df_roi=pileup_Unedit_day6_df_roi, out_dir=pileup_data_folder_path)
df_roi_stats


# #TODO:
- try MANUAL mC threshold selection for pileup??? set to 0.995 instead of 0.76 that was automatically selected?

# dmr modkit CRoff vs Unedited (NT)

3. Detecting differential modification at single base positions
The modkit dmr pair command has the ability to score individual bases (e.g. differentially methylated CpGs). To run single-base analysis on one or more paired samples, simply omit the --regions (-r) option when running modkit dmr pair. When performing single-base analysis the likelihood ratio score and a MAP-based p-value are available. For details on the likelihood ratio score and the MAP-based p-value, see the scoring details section. For example the above command becomes:

dmr_result=single_base_haplotype_dmr.bed

modkit dmr pair \
  -a ${hp1_pileup}.gz \
  -b ${hp2_pileup}.gz \
  -o ${dmr_result} \
  --ref ${ref} \
  --base C \
  --threads ${threads} \
  --log-filepath dmr.log

In [ ]:
# pileup_data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/unedited/merged_2libraries/dimelo_v2_output/new_pileup/"


In [ ]:
%%bash

date_today="20251110"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/unedited/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_pileup/"
# mkdir ${pileup_data_folder_path}

filtered_Unedit_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_day6_unedited_Tcells_mC0.995_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10.bam"
pileup_Unedit_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_NT_Day6_Tcells.bed"
cat "$pileup_Unedit_day6_bed"

# "/home/michalula/data/cas9_nanopore/data/20250721_nCATs_Tcells_UNEDITED_Day28/merged_outputs/5mCG/to_t2t_v2_0/pileup/pileup_Unedit_Day28_Tcells_20250721.bam"

echo "pileup_Unedit_day6_bed: ${pileup_Unedit_day6_bed}.gz"

ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_Unedit_day6_bed}.gz"
# ls -l "${pileup_Unedit_day28_bam}.gz"

chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${pileup_Unedit_day6_bed}.gz"

# ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_Unedit_day6_bed}.gz"


In [ ]:
%%bash
date_today="20251110"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_pileup/"
mkdir ${pileup_data_folder_path}
# Pre-filtered mC > 70%::
CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_day6_CRoff_Tcells_mC0.7_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10.bam"
pileup_CROFF_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

# data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/replica_1/analyze_single_reads/dimelo_v2_output/"
# pileup_data_folder_path=${data_folder_path}"new_pileup/"
# # mkdir ${pileup_data_folder_path}

# # Pre-filtered mC > 70%::
# CROFF_day6_bam=${data_folder_path}"pre_filtered_ROI_reads_Tcells_CRISPRoff_Day6_postEP_R9minion_threshold_mC0.7_T2Tv2_0_filterMode10_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60.bam"
# pileup_CROFF_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"


echo "pileup_CROFF_day6_bed: ${pileup_CROFF_day6_bed}.gz"

ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day6_bed}.gz"
# ls -l "${pileup_Unedit_day28_bam}.gz"

chmod u+rwx "${pileup_data_folder_path}"
chmod u+rwx "${pileup_CROFF_day6_bed}.gz"


# ls "${pileup_data_folder_path}"
ls -ld "${pileup_data_folder_path}"
ls -ld "${pileup_CROFF_day6_bed}.gz"


In [ ]:
%%bash
 
# 3. Detecting differential modification at single base positions
# The modkit dmr pair command has the ability to score individual bases (e.g. differentially methylated CpGs). To run single-base analysis on one or more paired samples, simply omit the --regions (-r) option when running modkit dmr pair. When performing single-base analysis the likelihood ratio score and a MAP-based p-value are available. For details on the likelihood ratio score and the MAP-based p-value, see the scoring details section. For example the above command becomes:

date_today="20251110"

experiment_codition="filtered_day6_CRoff_vs_Unedit"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/modkit_dmr/mc_07_filtered/new_dmr_output/"
dmr_result=${dmr_output_path}${date_today}"_single_base_noFiltered_mC07_"${experiment_codition}".bed"


data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/unedited/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_pileup/"
# mkdir ${pileup_data_folder_path}

filtered_Unedit_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_day6_unedited_Tcells_mC0.995_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10.bam"
pileup_Unedit_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_NT_Day6_Tcells.bed"
cat "$pileup_Unedit_day6_bed"



echo "pileup_Unedit_day6_bed: ${pileup_Unedit_day6_bed}.gz"
ls -l "${pileup_Unedit_day6_bed}.gz"

data_folder_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/croff/analyze_single_reads/dimelo_v2_output/"
pileup_data_folder_path=${data_folder_path}"new_pileup/"
mkdir ${pileup_data_folder_path}
# Pre-filtered mC > 70%::
CROFF_day6_bam=${data_folder_path}"filtered_reads_overlap_MORE_than_0.9_day6_CRoff_Tcells_mC0.7_T2Tv2_NoFullyUnmeth_ovrlap0.9_mismat0.7_mapQ60_modBaseQ10.bam"
pileup_CROFF_day6_bed=${pileup_data_folder_path}${date_today}"_filtered_mC07""_pileup_CROFF_Day6_Tcells.bed"

echo "pileup_CROFF_day6_bed: ${pileup_CROFF_day6_bed}.gz"
ls -l "${pileup_CROFF_day6_bed}.gz"

ref_genome_fa="/home/michalula/data/ref_genomes/t2t_v2_0/chm13v2.0.fa"
# ref="/home/michalula/data/ref_genomes/to_t2t_v1_1/chm13.draft_v1.1.fasta"
threads=32
 
cd ${dmr_output_path}
# '/home/michalula/code/epiCausality/epiCode/differential_methyl'

modkit dmr pair \
  -a ${pileup_CROFF_day6_bed}.gz \
  -b ${pileup_Unedit_day6_bed}.gz \
  -o ${dmr_result} \
  --ref ${ref_genome_fa} \
  --base C \
  --threads ${threads} \
  --log-filepath dmr.log


echo "dmr_result: $dmr_result"
ls -lah $dmr_result

In [ ]:
%%bash
date_today="20251110"

experiment_codition="filtered_day6_CRoff_vs_Unedit" 
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/modkit_dmr/mc_07_filtered/new_dmr_output/"
#  20251110_single_base_noFiltered_mC07_filtered_day6_CRoff_vs_Unedit.bed
"/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/modkit_dmr/mc_07_filtered/new_dmr_output/"
dmr_result=${dmr_output_path}${date_today}"_single_base_noFiltered_mC07_"${experiment_codition}".bed"

# date_today="20251109"
# experiment_codition="day6_CRoff_vs_Unedit"
# dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/modkit_dmr/mc_07_filtered/new_dmr_output/"
# dmr_result=${dmr_output_path}"_"${date_today}"single_base_filtered_mC07_"${experiment_codition}".bed"

echo "dmr_result: $dmr_result"
# ls -lah $dmr_result
ls -lah $dmr_output_path
# cat $dmr_result

In [ ]:
pwd

## modkit dmr explore output

The full table when performing single-site analysis with equal numbers of samples in groups, when running modkit dmr pair, will have the following schema:

column	name	description	type
1	chrom	name of reference sequence from bedMethyl input samples	str
2	start position	0-based start position, from --regions argument	int
3	end position	0-based exclusive end position, from --regions argument	int
4	name	name column from --regions BED, or chr:start-stop if absent, "." for single sites	str
5	score	difference score, more positive values have increased difference	float
6	strand	strand for the region or single-base position	str
7	samplea counts	counts of each base modification in the region, comma-separated, for sample A	str
8	samplea total	total number of base modification calls in the region, including unmodified, for sample A	int
9	sampleb counts	counts of each base modification in the region, comma-separated, for sample B	str
10	sampleb total	total number of base modification calls in the region, including unmodified, for sample B	int
11	samplea percents	percent of calls for each base modification in the region, comma-separated, for sample A	str
12	sampleb percents	percent of calls for each base modification in the region, comma-separated, for sample B	str
13	samplea fraction modified	fraction modification (of any kind) in sample A	float
14	sampleb fraction modified	fraction modification (of any kind) in sample B	float
15	MAP-based p-value	ratio of the posterior probability of observing the effect size over zero effect size	float
16	effect size	percent modified in sample A (col 12) minus percent modified in sample B (col 13)	float
<!-- 17	balanced MAP-based p-value	MAP-based p-value when all replicates are balanced	float
18	balanced effect size	effect size when all replicates are balanced	float -->


17	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
18	cohen_h_low	95% confidence interval lower bound	float
19	cohen_h_high	95% confidence interval upper bound	float

<!-- Differential methylation output format
The output from modkit dmr pair (and for each pairwise comparison with modkit dmr multi) is (roughly) a BED file with the following schema: -->
<!-- 
column	name	description	type
        1	chrom	name of reference sequence from bedMethyl input samples	str
        2	start position	0-based start position, from --regions argument	int
        3	end position	0-based exclusive end position, from --regions argument	int
        4	name	name column from --regions BED, or chr:start-stop if absent, "." for single sites	str
        5	score	difference score, more positive values have increased difference	float
        6	strand	strand for the region or single-base position	str
        7	samplea counts	counts of each base modification in the region, comma-separated, for sample A	str
        8	samplea total	total number of base modification calls in the region, including unmodified, for sample A	int
        9	sampleb counts	counts of each base modification in the region, comma-separated, for sample B	str
        10	sampleb total	total number of base modification calls in the region, including unmodified, for sample B	int
        11	samplea percents	percent of calls for each base modification in the region, comma-separated, for sample A	str
        12	sampleb percents	percent of calls for each base modification in the region, comma-separated, for sample B	str
        13	samplea fraction modified	fraction modification (of any kind) in sample A	float
        14	sampleb fraction modified	fraction modification (of any kind) in sample B	float
        15	MAP-based p-value	ratio of the posterior probability of observing the effect size over zero effect size	float
        16	effect size	percent modified in sample A (col 12) minus percent modified in sample B (col 13)	float
        17	balanced MAP-based p-value	MAP-based p-value when all replicates are balanced	float
        18	balanced effect size	effect size when all replicates are balanced	float
        19	pct_a_samples	percent of 'a' samples used in statistical test	float
        20	pct_b_samples	percent of 'b' samples used in statistical test	float
        21	per-replicate p-values	MAP-based p-values for matched replicate pairs	float
        22	per-replicate effect sizes	effect sizes matched replicate pairs	float
        23	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
        24	cohen_h_low	95% confidence interval lower bound	float
        25	cohen_h_high	95% confidence interval upper bound	float
        Columns 16-19 are only produced when multiple samples are provided, columns 20 and 21 are only produced when there is an equal number of 'a' and 'b' samples. When using multiple samples, it is possible that not every sample will have a modification fraction at a position. When this happens, the statistical test is still performed and the values of pct_a_samples and pct_b_samples reflect the percent of samples from each condition used in the test. 


     (15)	cohen_h	Cohen's h statistic (useful with regions and high-depth runs)	float
    (16)	cohen_h_low	95% confidence interval lower bound	float
    (17)	cohen_h_high	95% confidence interval upper bound	float
    
    n.b. Columns 15, 16, and 17 are present when the --regions option is passed, but these columns are on the right side of the table when performing single-site analysis (below). It is generally recommended to use the --header flag and standard CSV parsing to make sure the schema's between experiments are maintained.

When performing single-site analysis, the following additional columns are added:

column	name	description	type
Columns 20 and 21 have the replicate pairwise MAP-based p-values and effect sizes which are calculated based on their order provided on the command line. For example in the abbreviated command below:

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

In [ ]:
# dmr_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/modkit_dmr/mc_07_filtered/new_dmr_output/20251109_single_base_filtered_mC07_day6_CRoff_vs_Unedit.bed"

date_today="20251110"

experiment_codition="filtered_day6_CRoff_vs_Unedit"
dmr_output_path="/home/michalula/code/epiCausality/epiCode/analyze_ont_data/T2T_v2.0_mapped/T_cells/day_6/modkit_dmr/mc_07/filtered/new_dmr_output/"
dmr_result=dmr_output_path+date_today+"_single_base_noFiltered_mC07_"+experiment_codition+".bed"

dmr_path=dmr_result
dmr_path


In [ ]:
dmr_path

In [ ]:
out_dir = dmr_output_path
print("out_dir:", out_dir)

# Read DMR BED (robust to header/no-header) and assign canonical column names (uses existing vars: dmr_path, out_dir, date_today, pd, os)
canonical_cols = [
    "chrom", "start", "end", "name", "score", "strand",
    "samplea_counts", "samplea_total", "sampleb_counts", "sampleb_total",
    "samplea_percents", "sampleb_percents",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "cohen_h", "cohen_h_low", "cohen_h_high",
]
    # "balanced_map_pvalue", "balanced_effect_size"

# read file with header and fallback to header=None when headers look numeric or columns are unexpected
try:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

    # dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", engine="python") # , header=0
    # # heuristic: if too many numeric-looking column names, re-read as headerless
    # numeric_headers = sum(1 for c in dmr_df.columns if str(c).strip().isdigit())
    # if numeric_headers >= (len(dmr_df.columns) / 2) or dmr_df.shape[1] < 3:
    #     dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")
except Exception:
    dmr_df = pd.read_csv(dmr_path, sep="\t", comment="#", header=None, engine="python")

# assign canonical names up to number of columns present, add generic names for extras
ncols = dmr_df.shape[1]
if ncols <= len(canonical_cols):
    dmr_df.columns = canonical_cols[:ncols]
else:
    extras = [f"col_{i}" for i in range(ncols - len(canonical_cols))]
    dmr_df.columns = canonical_cols + extras

# coerce obvious numeric columns to numeric where present
num_cols_to_try = [
    "start", "end", "score",
    "samplea_total", "sampleb_total",
    "samplea_fraction_modified", "sampleb_fraction_modified",
    "map_pvalue", "effect_size",
    "balanced_map_pvalue", "balanced_effect_size"
]
for c in num_cols_to_try:
    if c in dmr_df.columns:
        dmr_df[c] = pd.to_numeric(dmr_df[c], errors="coerce")

# ensure output directory exists and save parsed table (parquet preferred)
os.makedirs(out_dir, exist_ok=True)
parsed_path = os.path.join(out_dir, f"{date_today}_dmr_parsed.parquet")
try:
    dmr_df.to_parquet(parsed_path, index=False)
    print("Saved parquet:", parsed_path)
except Exception:
    csv_path = os.path.join(out_dir, f"{date_today}_dmr_parsed.csv")
    dmr_df.to_csv(csv_path, index=False)
    print("Parquet not available, saved csv:", csv_path)

print("Loaded DMR:", dmr_path)
print("Assigned columns:", dmr_df.columns.tolist())
print("Shape:", dmr_df.shape)
dmr_df.head()

In [ ]:
import os
from IPython.display import display, HTML

# Visualize all columns from dmr_df and save interactive HTMLs to out_dir
import plotly.express as px
import plotly.graph_objects as go

os.makedirs(out_dir, exist_ok=True)

# Save a table summary
summary = dmr_df.describe(include='all').transpose()
summary_path = os.path.join(out_dir, f"{date_today}_dmr_column_summary.csv")
summary.to_csv(summary_path)

numcols = dmr_df.select_dtypes(include=['number']).columns.tolist()

def _safe_name(name):
    return str(name).replace(os.sep, "_").replace(" ", "_").replace("\t", "_")

# Per-column visualizations
for col in dmr_df.columns:
    safe = _safe_name(col)
    try:
        if col in numcols:
            # Histogram
            fig_h = px.histogram(dmr_df, x=col, nbins=80, title=f"Histogram: {col}")
            # fig_h.write_html(os.path.join(out_dir, f"{date_today}_dmr_hist_{safe}.html"), include_plotlyjs='cdn')
            fig_h.show()

            # Boxplot
            fig_b = px.box(dmr_df, y=col, points="outliers", title=f"Boxplot: {col}")
            # fig_b.write_html(os.path.join(out_dir, f"{date_today}_dmr_box_{safe}.html"), include_plotlyjs='cdn')
            fig_b.show()
        else:
            # Categorical / text: show top value counts (up to 50)
            vc = dmr_df[col].fillna("NA").astype(str).value_counts().head(50)
            if len(vc):
                fig_c = px.bar(x=vc.values[::-1], y=vc.index.astype(str)[::-1], orientation='h',
                               title=f"Top value counts: {col}", labels={'x':'count','y':col})
                fig_c.update_layout(yaxis={'categoryorder':'array','categoryarray':vc.index[::-1].astype(str).tolist()})
                # fig_c.write_html(os.path.join(out_dir, f"{date_today}_dmr_valcounts_{safe}.html"), include_plotlyjs='cdn')
                fig_c.show()
            else:
                # fallback: display empty info
                display(HTML(f"<b>{col}</b>: no values to plot"))
    except Exception as e:
        print(f"Skipped plotting column {col!r} due to error: {e}")

# Correlation heatmap for numeric columns
if len(numcols) >= 2:
    try:
        corr = dmr_df[numcols].corr()
        fig_corr = px.imshow(corr, text_auto=True, aspect="auto", title="Correlation matrix (numeric columns)")
        # fig_corr.write_html(os.path.join(out_dir, f"{date_today}_dmr_correlation_numeric.html"), include_plotlyjs='cdn')
        fig_corr.show()
    except Exception as e:
        print("Failed to create correlation heatmap:", e)

print("Saved summary:", summary_path)
print("Plots saved to:", out_dir)

In [ ]:
# Select significant CG pairs from DMR results and plot them (new cell at index 69).
# Uses existing notebook variables: dmr_df (parsed modkit dmr), df_roi_stats (pileup ROI stats),
# plotly (px) and out_dir/dmr_folder_path for saving. Does not re-import modules.

# Parameters
pvalue_thresh = 0.05

# ensure numeric columns
dmr_df['map_pvalue'] = pd.to_numeric(dmr_df['map_pvalue'], errors='coerce')
dmr_df['effect_size'] = pd.to_numeric(dmr_df['effect_size'], errors='coerce')
dmr_df['samplea_fraction_modified'] = pd.to_numeric(dmr_df['samplea_fraction_modified'], errors='coerce')
dmr_df['sampleb_fraction_modified'] = pd.to_numeric(dmr_df['sampleb_fraction_modified'], errors='coerce')

# filter significant by MAP-based p-value
sig = dmr_df[dmr_df['map_pvalue'] <= pvalue_thresh].copy()

dmr_df['map_pval_less005'] = dmr_df['map_pvalue'] <= 0.05


# # restrict to ROI positions if df_roi_stats exists
# if 'df_roi_stats' in globals():
#     roi_positions = set(df_roi_stats['start'].astype(int).tolist())
#     sig = sig[sig['start'].isin(roi_positions)].copy()

# quick exit if none
if sig.shape[0] == 0:
    print(f"No significant CG pairs found in ROI at map_pvalue <= {pvalue_thresh}")
else:
    # add convenience cols
    sig['pos'] = sig['start'].astype(str)
    sig['a_perc'] = sig['samplea_fraction_modified'] * 100
    sig['b_perc'] = sig['sampleb_fraction_modified'] * 100
    sig['total_reads'] = sig.get('samplea_total', 0).fillna(0).astype(int) + sig.get('sampleb_total', 0).fillna(0).astype(int)

    # save a table of significant sites
    os.makedirs(out_dir, exist_ok=True)
    sig_table_path = os.path.join(out_dir, f"dmr_significant_p{pvalue_thresh:.3f}_roi.tsv")
    sig.to_csv(sig_table_path, sep='\t', index=False)
    print("Saved significant sites table:", sig_table_path)
    display(sig[['chrom','start','end','strand','map_pvalue','effect_size','a_perc','b_perc','total_reads']].reset_index(drop=True))

    # plot the map_pval_less005 distribution which corresponds to significant sites
    fig_mappval_hist = px.histogram(
        dmr_df,
        x='map_pval_less005',
        nbins=80,
        title=f"MAP-based p-value distribution (highlighting p <= {pvalue_thresh}) <br>{experiment_codition}",
        labels={'map_pval_less005':'MAP-based p-value'}
    )
    fig_mappval_hist.update_layout(height=520)
    mappval_hist_path = os.path.join(out_dir, f"dmr_map_pval_distribution.html")
    # fig_mappval_hist.write_html(mappval_hist_path, include_plotlyjs='cdn')
    fig_mappval_hist.show()
    # print("Saved MAP-based p-value distribution histogram:", mappval_hist_path)

    # plot the percent of significant sites where map_pvalue <= pvalue_thresh is colored red, others blue (color not working)
    # Check https://plotly.com/python/pie-charts/ for coloring instructions
    percent_significant = (sig.shape[0] / dmr_df.shape[0]) * 100
    fig_mappval_pie = px.pie(
        dmr_df,
        names=['Not Significant (p > {})'.format(pvalue_thresh), 'Significant (p <= {})'.format(pvalue_thresh)],
        values=[dmr_df.shape[0] - sig.shape[0],sig.shape[0]],
        title=f"Percentage of significant CGs (map_pvalue <= {pvalue_thresh}): {percent_significant:.2f}% <br>{experiment_codition}",
        # color_discrete_map={'Not Significant (p > {})'.format(pvalue_thresh): 'blue',
        #                     'Significant (p <= {})'.format(pvalue_thresh): 'red'},
        # colors=['blue','red']
    )
    fig_mappval_pie.update_layout(height=520)
    mappval_pie_path = os.path.join(out_dir, f"dmr_map_pval_percentage.html")
    # fig_mappval_pie.write_html(mappval_pie_path, include_plotlyjs='cdn')
    fig_mappval_pie.show()
    # print("Saved MAP-based p-value percentage pie chart:", mappval_pie_path)


    # plot effect size distribution of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh
    fig_effectsize_hist = px.histogram(
        dmr_df,
        x='effect_size',
        nbins=80,
        # add a line break and write experiment_codition into the title
        title=f"Effect size distribution (highlighting significant sites with map_pvalue <= {pvalue_thresh})<br>{experiment_codition}",
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        color_discrete_map={True: 'red', False: 'blue'},
    )
            # labels={'effect_size':'Effect size (A - B)'}

    fig_effectsize_hist.update_layout(height=520)
    effectsize_hist_path = os.path.join(out_dir, f"dmr_effect_size_distribution.html")
    # fig_effectsize_hist.write_html(effectsize_hist_path, include_plotlyjs='cdn')
    fig_effectsize_hist.show()
    # print("Saved effect size distribution histogram:", effectsize_hist_path)        

    # plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    # add color legend names as 'Significant: map_pvalue <= pvalue_thresh' and 'Not Significant: map_pvalue > pvalue_thresh'  
    fig_effectsize_scatter = px.scatter(
        dmr_df,
        x=dmr_df.index,
        y='effect_size',
        color_discrete_map={True: 'red', False: 'blue'},
        color=(dmr_df['map_pvalue'] <= pvalue_thresh), 
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        title=f"Effect sizes for all CGs (highlighting significant sites with map_pvalue <= {pvalue_thresh}) <br>{experiment_codition}",
    )
    fig_effectsize_scatter.update_layout(height=520)
    effectsize_scatter_path = os.path.join(out_dir, f"dmr_effect_size_scatter.html")
    # fig_effectsize_scatter.write_html(effectsize_scatter_path, include_plotlyjs='cdn')
    fig_effectsize_scatter.show()
    # print("Saved effect size scatter plot:", effectsize_scatter_path)       

    # bar plot effect sizes of all sites without sorting and highlight significant which have map_pvalue <= pvalue_thresh 
    fig_effectsize_bar = px.bar(
        dmr_df,
        x=dmr_df.index,         
        y='effect_size',
        color=(dmr_df['map_pvalue'] <= pvalue_thresh),
        labels={'effect_size':'Effect size (A - B)','index':'Index',
                'color':f'Significant: map_pvalue <= {pvalue_thresh}'},
        color_discrete_map={True: 'red', False: 'blue'},
        title=f"Effect sizes for all CGs (n={len(dmr_df)}) (highlighting significant sites with map_pvalue <= {pvalue_thresh}) <br>{experiment_codition}",
    )
    fig_effectsize_bar.update_layout(height=520)
    effectsize_bar_path = os.path.join(out_dir, f"dmr_effect_size_bar.html")
    # fig_effectsize_bar.write_html(effectsize_bar_path, include_plotlyjs='cdn')
    fig_effectsize_bar.show()
    # print("Saved effect size bar plot:", effectsize_bar_path) 


    # Bar: effect size per position (without sorting) with effect size colors 
    # dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
    fig_bar_unsorted = px.bar(
        dmr_df,
        x=dmr_df.index, 
        y='effect_size',        
        color='effect_size',
        title=f"Effect size for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
            # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
            # x='label',

    fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
    bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
    # fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
    fig_bar_unsorted.show()
    # print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
        
    # Bar: effect size per position (without sorting)
    sig['label'] = sig['pos'] + ":" + sig['strand'].astype(str)
    fig_bar_unsorted = px.bar(
        sig,
        x='label',
        y='effect_size',        
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig)}) with map_pvalue <= {pvalue_thresh} <br>{experiment_codition}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
    bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
    # fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
    fig_bar_unsorted.show()
    # print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
        

    # Bar: effect size per position (sorted)
    sig_sorted = sig.sort_values('effect_size', ascending=False).copy()
    sig_sorted['label'] = sig_sorted['pos'] + ":" + sig_sorted['strand'].astype(str)
    fig_bar = px.bar(
        sig_sorted,
        x='label',
        y='effect_size',
        color='effect_size',
        hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        title=f"Effect size for significant CGs (n={len(sig_sorted)}) <br>{experiment_codition}",
        labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
    )
    fig_bar.update_layout(xaxis_tickangle=45, height=520)
    bar_path = os.path.join(out_dir, f"dmr_sig_effectsize_p{pvalue_thresh:.3f}.html")
    # fig_bar.write_html(bar_path, include_plotlyjs='cdn')
    fig_bar.show()
    # print("Saved effect-size bar plot:", bar_path)



    # Scatter: sample A vs sample B percent modified (size = total reads, color = effect size)
    fig_scatter = px.scatter(
        sig,
        x='a_perc',
        y='b_perc',
        color='effect_size',
        size='total_reads',
        hover_data=['pos','start','map_pvalue','effect_size','cohen_h'],
        title=f"Significant CGs (map_pvalue <= {pvalue_thresh}) — sample A vs B percent modified <br>{experiment_codition}",
        labels={'a_perc':'Sample A % modified','b_perc':'Sample B % modified'}
    )
    fig_scatter.update_layout(height=520)
    scatter_path = os.path.join(out_dir, f"dmr_sig_scatter_p{pvalue_thresh:.3f}.html")
    # fig_scatter.write_html(scatter_path, include_plotlyjs='cdn')
    fig_scatter.show()
    # print("Saved scatter plot:", scatter_path)


In [ ]:
import numpy as np

dmr_df['map_pval_log10'] = -np.log10(dmr_df['map_pvalue'] + 1e-10)  # add small constant to avoid log(0)
# dmr_df

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='effect_size',        
    color='map_pvalue',
    title=f"Effect size & p-vals for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='effect_size',        
    color='map_pval_log10',
    title=f"Effect size & log(p-vals) for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='effect_size',        
    color='map_pval_less005',
    title=f"Effect size & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='score',        
    color='map_pval_less005',
    title=f"Effect size & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score':'LMR','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:
cg_units_pair = [[f'CG_{i}', f'CG_{i}'] for i in range(-1,142)]
cg_units_pair = np.concat(cg_units_pair)

dmr_df['cg_unit_pair'] = cg_units_pair
dmr_df

In [ ]:
cg_units_pair_id = [[f'CG_{i}.+', f'CG_{i}.-'] for i in range(-1,142)]
cg_units_pair_id = np.concat(cg_units_pair_id)
cg_units_pair_id

In [ ]:
dmr_df.shape, cg_units_pair_id.shape

In [ ]:
dmr_df['cg_unit_pair_id'] = cg_units_pair_id
dmr_df

In [ ]:
dmr_df[['cg_unit_pair_id']]

In [ ]:
experiment_codition

In [ ]:

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='effect_size',        
    color='map_pval_less005',
    title=f"Effect size & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    
    
# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='score',        
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score':'LML','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:
# Overlap the two barplots: effect_size (primary y) and score/LMR (secondary y)
# Uses existing notebook variables: dmr_df, out_dir, pvalue_thresh, experiment_codition, go, os

# prepare x (use index so bars align with previous plots)
x = dmr_df.index

# color effect_size by significance boolean already present
effect_colors = dmr_df['map_pval_less005'].map({True: 'red', False: 'steelblue'}).tolist()

trace_effect = go.Bar(
    x=x,
    y=dmr_df['effect_size'],
    name='Effect size (A - B)',
    marker=dict(color=effect_colors),
    opacity=0.85,
    hovertemplate='Index=%{x}<br>Effect size=%{y:.3f}<br>p<=0.05=%{marker.color}<extra></extra>',
    yaxis='y1'
)

trace_score = go.Bar(
    x=x,
    y=dmr_df['score'],
    name='Score (LMR)',
    marker=dict(color='green'),
    opacity=0.6,
    hovertemplate='Index=%{x}<br>Score=%{y:.3f}<extra></extra>',
    yaxis='y2'
)

fig = go.Figure(data=[trace_score, trace_effect])

fig.update_layout(
    title=f"Overlay: Effect size and Score (LMR) — {experiment_codition} (n={len(dmr_df)})",
    xaxis=dict(title='Index', tickangle=45),
    yaxis=dict(title='Effect size (A - B)', side='left', showgrid=True),
    yaxis2=dict(title='Score (LMR)', overlaying='y', side='right', showgrid=False),
    barmode='overlay',
    height=540,
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

# save and display
os.makedirs(out_dir, exist_ok=True)
# outpath = os.path.join(out_dir, f"dmr_overlay_effectsize_score_p{pvalue_thresh:.3f}.html")
# fig.write_html(outpath, include_plotlyjs='cdn')
# print("Saved overlay plot to:", outpath)
fig.show()

In [ ]:
dmr_df['efsize-score'] = dmr_df['effect_size'] - dmr_df['score']
dmr_df

In [ ]:
dmr_df['score-efsize'] = dmr_df['score'] - dmr_df['effect_size']
dmr_df

In [ ]:
# min-max scale 'score' to [0,1] and update overlay figure if present
s_min = dmr_df['score'].min(skipna=True)
s_max = dmr_df['score'].max(skipna=True)

if pd.isna(s_min) or pd.isna(s_max):
    dmr_df['score_scaled'] = pd.NA
else:
    if s_max == s_min:
        dmr_df['score_scaled'] = 0.0
    else:
        dmr_df['score_scaled'] = (dmr_df['score'] - s_min) / (s_max - s_min)

print(f"score min={s_min}, max={s_max} -> added column 'score_scaled' (0-1)")

# If overlay figure exists, update the score trace to use the scaled values and adjust right y-axis
if 'fig' in globals() and len(fig.data) > 0:
    try:
        # in this notebook fig.data[0] is the score trace
        fig.data[0].y = dmr_df['score_scaled'].astype(float).values
        fig.data[0].name = 'Score (LMR) (scaled 0-1)'
        # adjust right y-axis to [0,1]
        if 'yaxis2' in fig.layout:
            fig.layout.yaxis2.title.text = 'Score (LMR) (scaled 0-1)'
            fig.layout.yaxis2.range = [0, 1]
        else:
            fig.update_layout(yaxis2=dict(title='Score (LMR) (scaled 0-1)', overlaying='y', side='right', range=[0,1]))
        fig.show()
    except Exception as e:
        print("Failed to update fig with scaled scores:", e)

# show a quick check
display(dmr_df[['score','score_scaled']].head())

In [ ]:
dmr_df['efsize-score_scaled'] = dmr_df['effect_size'] - dmr_df['score_scaled'] 
dmr_df

In [ ]:

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='efsize-score_scaled',        
    color='map_pval_less005',
    title=f"sefsize-score_scaled (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'efsize-score_scaled':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:
dmr_df.columns

In [ ]:

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df['start'], 
    y='effect_size',        
    color='map_pval_less005',
    title=f"sefsize-score_scaled (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'efsize-score_scaled':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:
# Plot effect_size with x-axis = 'start' (uses existing notebook variables: dmr_df, out_dir, pvalue_thresh, experiment_codition)
df_plot = dmr_df.copy()
df_plot['start'] = pd.to_numeric(df_plot['start'], errors='coerce')
df_plot['effect_size'] = pd.to_numeric(df_plot['effect_size'], errors='coerce')
df_plot = df_plot.sort_values('start')

fig = px.bar(
    df_plot,
    x='start',
    y='effect_size',
    color='map_pval_less005',
    color_discrete_map={True: 'red', False: 'steelblue'},
    title=f"Effect size by genomic start — {experiment_codition}",
    labels={'start': 'Genomic start', 'effect_size': 'Effect size (A - B)', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)
fig.update_layout(height=520, xaxis_tickangle=45)

outpath = os.path.join(out_dir, "dmr_effect_size_by_start.html")
try:
    fig.write_html(outpath, include_plotlyjs='cdn')
    print("Saved plot to:", outpath)
except Exception:
    print("Could not save HTML, displaying inline instead.")
fig.show()

In [ ]:
# Plot effect_size with x-axis = 'start' and show y-values as labels on bars (cell 95).
# Uses existing notebook variables: dmr_df, out_dir, experiment_codition

# prepare dataframe
df_plot = dmr_df.copy()
df_plot['start'] = pd.to_numeric(df_plot['start'], errors='coerce')
df_plot['effect_size'] = pd.to_numeric(df_plot['effect_size'], errors='coerce')
df_plot = df_plot.sort_values('start').reset_index(drop=True)

# text labels for bars (rounded to 3 decimals)
text_vals = df_plot['effect_size'].round(3).astype(str)

# create bar plot and include text labels
fig = px.bar(
    df_plot,
    x='start',
    y='effect_size',
    color='map_pval_less005',
    color_discrete_map={True: 'red', False: 'steelblue'},
    text=text_vals,
    title=f"Effect size by genomic start — {experiment_codition}",
    labels={'start': 'Genomic start', 'effect_size': 'Effect size (A - B)'}
)

# show text outside bars and improve readability
fig.update_traces(textposition='outside', textfont=dict(size=9), cliponaxis=False)

# add vertical padding so outside labels are visible
ymin = df_plot['effect_size'].min(skipna=True)
ymax = df_plot['effect_size'].max(skipna=True)
if pd.notna(ymin) and pd.notna(ymax):
    pad = max(0.05, (ymax - ymin) * 0.08)  # at least a small pad
    fig.update_yaxes(range=[ymin - pad, ymax + pad])

# layout tweaks for readability
fig.update_layout(
    height=700,
    xaxis_tickangle=45,
    uniformtext_minsize=6,
    uniformtext_mode='hide',
    margin=dict(t=80, b=200)
)

# save and show
# outpath = os.path.join(out_dir, "dmr_effect_size_by_start_with_labels.html")
# try:
#     fig.write_html(outpath, include_plotlyjs='cdn')
#     print("Saved plot to:", outpath)
# except Exception:
#     print("Could not save HTML, displaying inline instead.")
fig.show()

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(dmr_df['start'], dmr_df['effect_size'])

In [ ]:
plt.bar(dmr_df['start'], dmr_df['effect_size'])

In [ ]:
plt.bar(dmr_df['start'], dmr_df['score'])

# Add Hg38 coords by shifting:
-734704

In [ ]:
dmr_df['start_hg38']= dmr_df['start'] + 734704
dmr_df['end_hg38']= dmr_df['end'] + 734704
dmr_df['start_t2tv2']= dmr_df['start'] #+ 734d704
dmr_df['end_t2tv2']= dmr_df['end'] #+ 734704

In [ ]:
dmr_df.columns

In [ ]:
dmr_df['cg_unit_pair_id']

# lable x-ticks

In [ ]:
# create CG unit pair ids (and keep the user-typed alias with the typo)
# uses existing notebook variables: dmr_df, np, px, os, out_dir, experiment_codition, pvalue_thresh

# build labels based on sorted genomic position and strand, pairing consecutive entries
# order = dmr_df.sort_values(['start', 'strand']).index.to_numpy()
# cg_nums = (np.arange(len(order)) // 2) + 1

# labels = [None] * len(dmr_df)
# for i, orig_idx in enumerate(order):
#     strand = dmr_df.at[orig_idx, 'strand']
#     labels[orig_idx] = f"CG_{cg_nums[i]}.{strand}"

# dmr_df['cg_unit_pair_id'] = labels
# # add alias matching the user's requested (typo) name
# dmr_df['cg_unit_pait_id'] = dmr_df['cg_unit_pair_id']

# Bar: use cg_unit_pair_id on x-axis
fig_bar_unsorted = px.bar(
    dmr_df,
    x='cg_unit_pair_id',
    y='score',
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score': 'LML', 'cg_unit_pair_id': 'CG unit id', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520, margin=dict(b=200))
os.makedirs(out_dir, exist_ok=True)
outpath = os.path.join(out_dir, f"dmr_score_by_cg_unit.html")
try:
    fig_bar_unsorted.write_html(outpath, include_plotlyjs='cdn')
    print("Saved plot to:", outpath)
except Exception:
    print("Could not save HTML, displaying inline instead.")
fig_bar_unsorted.show()

In [ ]:

# Bar: use cg_unit_pair_id on x-axis
fig_bar_unsorted = px.bar(
    dmr_df,
    x='cg_unit_pair_id',
    y='effect_size',
    color='map_pval_less005',
    title=f"Effect size with p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score': 'Effect size', 'cg_unit_pair_id': 'CG unit id', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520, margin=dict(b=200))
os.makedirs(out_dir, exist_ok=True)
outpath = os.path.join(out_dir, f"dmr_effect_size_by_cg_unit.html")
try:
    fig_bar_unsorted.write_html(outpath, include_plotlyjs='cdn')
    print("Saved plot to:", outpath)
except Exception:
    print("Could not save HTML, displaying inline instead.")
fig_bar_unsorted.show()

In [ ]:
# Preserve original dataframe row order and plot cg_unit_pair_id in that exact order
# Uses existing notebook variables: dmr_df, out_dir, experiment_codition, pvalue_thresh, os, px

# create a copy that preserves current row order and ensures cg_unit_pair_id is categorical in that order
df_plot = dmr_df.copy().reset_index(drop=True)
ordered_cats = df_plot['cg_unit_pair_id'].astype(str).tolist()
df_plot['cg_unit_pair_id'] = pd.Categorical(df_plot['cg_unit_pair_id'], categories=ordered_cats, ordered=True)

fig_b = px.bar(
    df_plot,
    x='cg_unit_pair_id',
    y='effect_size',
    color='map_pval_less005',
    category_orders={'cg_unit_pair_id': ordered_cats},
    title=f"Effect size with p-vals<0.005 for all CGs (n={len(df_plot)}) <br>{experiment_codition}",
    labels={'score': 'Effect size', 'cg_unit_pair_id': 'CG unit id', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)

fig_b.update_layout(xaxis_tickangle=45, height=520, margin=dict(b=200))
os.makedirs(out_dir, exist_ok=True)
outpath = os.path.join(out_dir, f"dmr_effect_size_by_cgID_{experiment_codition}.html")
try:
    fig_b.write_html(outpath, include_plotlyjs='cdn')
    print("Saved plot to:", outpath)
except Exception:
    print("Could not save HTML, displaying inline instead.")
fig_b.show()

In [ ]:
# Preserve original dataframe row order and plot cg_unit_pair_id in that exact order
# Uses existing notebook variables: dmr_df, out_dir, experiment_codition, pvalue_thresh, os, px

# create a copy that preserves current row order and ensures cg_unit_pair_id is categorical in that order
# df_plot = dmr_df.copy().reset_index(drop=True)
# ordered_cats = df_plot['cg_unit_pair_id'].astype(str).tolist()
# df_plot['cg_unit_pair_id'] = pd.Categorical(df_plot['cg_unit_pair_id'], categories=ordered_cats, ordered=True)

fig_b = px.bar(
    dmr_df,
    x='start',
    y='effect_size',
    color='map_pval_less005',
    category_orders={'cg_unit_pair_id': ordered_cats},
    title=f"Effect size with p-vals<0.005 for all CGs (n={len(df_plot)}) <br>{experiment_codition}",
    labels={'score': 'Effect size', 'cg_unit_pair_id': 'CG unit id', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)

fig_b.update_layout(xaxis_tickangle=45, height=520, margin=dict(b=200))


# 🚀 Make bars visible
fig_b.update_traces(width=8, marker_line_width=0)

# Optional: larger plot
fig_b.update_layout(xaxis_tickangle=45, height=700, width=1600, margin=dict(b=200))


os.makedirs(out_dir, exist_ok=True)
outpath = os.path.join(out_dir, f"dmr_effect_size_genome_whide_{experiment_codition}.html")
try:
    fig_b.write_html(outpath, include_plotlyjs='cdn')
    print("Saved plot to:", outpath)
except Exception:
    print("Could not save HTML, displaying inline instead.")
fig_b.show()

In [ ]:
# Preserve original dataframe row order and plot cg_unit_pair_id in that exact order
# Uses existing notebook variables: dmr_df, out_dir, experiment_codition, pvalue_thresh, os, px

# create a copy that preserves current row order and ensures cg_unit_pair_id is categorical in that order
# df_plot = dmr_df.copy().reset_index(drop=True)
# ordered_cats = df_plot['cg_unit_pair_id'].astype(str).tolist()
# df_plot['cg_unit_pair_id'] = pd.Categorical(df_plot['cg_unit_pair_id'], categories=ordered_cats, ordered=True)

fig_b = px.bar(
    dmr_df,
    x='start',
    y='effect_size',
    opacity=0.6,
    color='map_pval_less005',
    category_orders={'cg_unit_pair_id': ordered_cats},
    title=f"Effect size with p-vals<0.005 for all CGs (n={len(df_plot)}) <br>{experiment_codition}",
    labels={'score': 'Effect size', 'cg_unit_pair_id': 'CG unit id', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)

fig_b.update_layout(xaxis_tickangle=45, height=520, margin=dict(b=200))


# 🚀 Make bars visible
fig_b.update_traces(width=8, marker_line_width=0)

# Optional: larger plot
# fig_b.update_layout(xaxis_tickangle=45, height=700, width=1600, margin=dict(b=200))


os.makedirs(out_dir, exist_ok=True)
outpath = os.path.join(out_dir, f"dmr_effect_size_genome_whide_{experiment_codition}.html")
try:
    fig_b.write_html(outpath, include_plotlyjs='cdn')
    print("Saved plot to:", outpath)
except Exception:
    print("Could not save HTML, displaying inline instead.")
fig_b.show()

In [ ]:
plt.scatter(dmr_df['start'], dmr_df['effect_size'])

In [ ]:

# # Bar: effect size per position (without sorting) with effect size colors 
# # dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
# fig_bar_unsorted = px.bar(
#     dmr_df,
#     x='cg_unit_pair_id',
#     y='efsize-score_scaled',        
#     color='map_pval_less005',
#     title=f"sefsize-score_scaled (n={len(dmr_df)}) <br>{experiment_codition}",
#     labels={'efsize-score_scaled':'Effect size (A - B)','label':'position:strand'}
# )
#         # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
#         # x='label',

# fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
# bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# # fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
# fig_bar_unsorted.show()
# # print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='efsize-score_scaled',        
    color='map_pval_less005',
    title=f"sefsize-score_scaled (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'efsize-score_scaled':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',
        # x='cg_unit_pair_id',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:
dmr_df["score_norm"]= (dmr_df["score"] - dmr_df["score"].min()) / (dmr_df["score"].max() - dmr_df["score"].min())
dmr_df["score_norm"]

In [ ]:
# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='score_norm',        
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score':'LML','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()

In [ ]:
plt.figure(figsize=(16, 5))

plt.bar(np.arange(len(dmr_df['effect_size'])), dmr_df['effect_size'], 
        snap=False, alpha=0.5, label='effect_size', color='green') #, color=dmr_df['map_pval_less005'])

plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label='score_norm', color='red') #, color=dmr_df['map_pval_less005'])

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Score') 
plt.title('Effect size & Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
plt.show()

In [ ]:
# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='score_norm',        
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score':'LML','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()

In [ ]:
# dmr_df = df_dmr_K562_ZFPoffHigh_vs_Unedit_filtered_mC07 
# experiment_codition="K562_ZFPoffHigh_vs_Unedit_filtered_mC07"
# out_dir=dmr_dir_path
# pvalue_thresh=0.005

plt.figure(figsize=(16, 5))
plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label=experiment_codition, color='red')
        # color by map_pval_less005

# plt.bar(np.arange(len(CGs_D6_CRoff_mc07_fracs)), CGs_D6_CRoff_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 CRISPRoff', color='deepskyblue')
# plt.bar(np.arange(len(CGs_D6_unedited_mc07_fracs)), CGs_D6_unedited_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 Unedited', color='navy')

# plt.bar(np.arange(len(CGs_D35_unedited_mc07_fracs)), CGs_D35_unedited_mc07_fracs, snap=False, alpha=0.5, label='D35 mC > 0.7 Unedited', color='red')
# mediumblue.'turquoise', 'aquamarine', 'deepskyblue', 'dodgerblue', 'cornflowerblue', 'royalblue', 'blue', 'mediumblue', 'darkblue', 'navy', 'midnightblue'

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Normalized LML Score') 
plt.title('Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
# Show the plot
plt.show()

In [ ]:
from matplotlib.patches import Patch

# color bars by boolean column 'map_pval_less005'
colors = dmr_df['map_pval_less005'].map({True: 'red', False: 'blue'}).tolist()

plt.figure(figsize=(16, 5))
x = np.arange(len(dmr_df))
y = dmr_df['score_norm']
plt.bar(x, y, color=colors, alpha=0.8, snap=False)
plt.xlabel('CpG units')
plt.ylabel('Normalized LML Score')
plt.title('Normalized LML Score per CpG \n' + experiment_codition)

plt.legend(handles=[Patch(color='red', label='map_pval_less005 = True'),
                    Patch(color='lightgray', label='map_pval_less005 = False')])
plt.show()

In [ ]:
dmr_df.columns

In [ ]:
plt.figure(figsize=(16, 5))

plt.bar(np.arange(len(dmr_df['effect_size'])), dmr_df['effect_size'], 
        snap=False, alpha=0.5, label='effect_size', color='green') #, color=dmr_df['map_pval_less005'])

plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label='score_norm', color='red') #, color=dmr_df['map_pval_less005'])

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Score') 
plt.title('Effect size & Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
plt.show()

In [ ]:
# dmr_df = df_dmr_K562_ZFPoffHigh_vs_Unedit_filtered_mC07 
# experiment_codition="K562_ZFPoffHigh_vs_Unedit_filtered_mC07"
# out_dir=dmr_dir_path
# pvalue_thresh=0.005

plt.figure(figsize=(16, 5))

plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label=experiment_codition) # , color=dmr_df['map_pval_less005'])
        # color by map_pval_less005

# plt.bar(np.arange(len(CGs_D6_CRoff_mc07_fracs)), CGs_D6_CRoff_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 CRISPRoff', color='deepskyblue')
# plt.bar(np.arange(len(CGs_D6_unedited_mc07_fracs)), CGs_D6_unedited_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 Unedited', color='navy')

# plt.bar(np.arange(len(CGs_D35_unedited_mc07_fracs)), CGs_D35_unedited_mc07_fracs, snap=False, alpha=0.5, label='D35 mC > 0.7 Unedited', color='red')
# mediumblue.'turquoise', 'aquamarine', 'deepskyblue', 'dodgerblue', 'cornflowerblue', 'royalblue', 'blue', 'mediumblue', 'darkblue', 'navy', 'midnightblue'

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Normalized LML Score') 
plt.title('Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
# Show the plot
plt.show()

In [ ]:


# Bar: use cg_unit_pair_id on x-axis
fig_bar_unsorted = px.bar(
    dmr_df,
    x='start_hg38',
    y='score',
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score': 'LML', 'cg_unit_pair_id': 'CG unit id', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520, margin=dict(b=200))

fig_bar_unsorted.show()

In [ ]:

# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='effect_size',        
    color='map_pval_less005',
    title=f"Effect size & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'effect_size':'Effect size (A - B)','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    
    
# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='score',        
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score':'LML','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()
# print("Saved unsorted effect-size bar plot:", bar_unsorted_path)    
    

In [ ]:
# Preserve original dataframe row order and plot cg_unit_pair_id in that exact order
# Uses existing notebook variables: dmr_df, out_dir, experiment_codition, pvalue_thresh, os, px

# create a copy that preserves current row order and ensures cg_unit_pair_id is categorical in that order
# df_plot = dmr_df.copy().reset_index(drop=True)
# ordered_cats = df_plot['cg_unit_pair_id'].astype(str).tolist()
# df_plot['cg_unit_pair_id'] = pd.Categorical(df_plot['cg_unit_pair_id'], categories=ordered_cats, ordered=True)

fig_b = px.bar(
    dmr_df,
    x='start',
    y='effect_size',
    opacity=0.3,
    color='map_pval_less005',
    category_orders={'cg_unit_pair_id': ordered_cats},
    title=f"Effect size with p-vals<0.005 for all CGs (n={len(df_plot)}) <br>{experiment_codition}",
    labels={'score': 'Effect size', 'cg_unit_pair_id': 'CG unit id', 'map_pval_less005': f'Significant (p <= {pvalue_thresh})'}
)

fig_b.update_layout(xaxis_tickangle=45, height=520, margin=dict(b=200))


# 🚀 Make bars visible
fig_b.update_traces(width=8, marker_line_width=0)

# Optional: larger plot
# fig_b.update_layout(xaxis_tickangle=45, height=700, width=1600, margin=dict(b=200))


os.makedirs(out_dir, exist_ok=True)
outpath = os.path.join(out_dir, f"dmr_effect_size_genome_whide_{experiment_codition}_opac03.html")
try:
    fig_b.write_html(outpath, include_plotlyjs='cdn')
    print("Saved plot to:", outpath)
except Exception:
    print("Could not save HTML, displaying inline instead.")
fig_b.show()

In [ ]:
dmr_df

In [ ]:
dmr_df.columns

In [ ]:
dmr_df["score_norm"]= (dmr_df["score"] - dmr_df["score"].min()) / (dmr_df["score"].max() - dmr_df["score"].min())
dmr_df["score_norm"]

In [ ]:
# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='score_norm',        
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score':'LML','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()

In [ ]:
plt.figure(figsize=(16, 5))

plt.bar(np.arange(len(dmr_df['effect_size'])), dmr_df['effect_size'], 
        snap=False, alpha=0.5, label='effect_size', color='green') #, color=dmr_df['map_pval_less005'])

plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label='score_norm', color='red') #, color=dmr_df['map_pval_less005'])

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Score') 
plt.title('Effect size & Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
plt.show()

In [ ]:
# Bar: effect size per position (without sorting) with effect size colors 
# dmr_df['label'] = dmr_df['pos'] + ":" + dmr_df['strand'].astype(str)
fig_bar_unsorted = px.bar(
    dmr_df,
    x=dmr_df.index, 
    y='score_norm',        
    color='map_pval_less005',
    title=f"log marginal Likelihood(LML) score & p-vals<0.005 for all CGs (n={len(dmr_df)}) <br>{experiment_codition}",
    labels={'score':'LML','label':'position:strand'}
)
        # hover_data=['map_pvalue','a_perc','b_perc','total_reads'],
        # x='label',

fig_bar_unsorted.update_layout(xaxis_tickangle=45, height=520)
bar_unsorted_path = os.path.join(out_dir, f"dmr_sig_effectsize_unsorted_p{pvalue_thresh:.3f}.html")
# fig_bar_unsorted.write_html(bar_unsorted_path, include_plotlyjs='cdn')
fig_bar_unsorted.show()

In [ ]:
# dmr_df = df_dmr_K562_ZFPoffHigh_vs_Unedit_filtered_mC07 
# experiment_codition="K562_ZFPoffHigh_vs_Unedit_filtered_mC07"
# out_dir=dmr_dir_path
# pvalue_thresh=0.005

plt.figure(figsize=(16, 5))
plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label=experiment_codition, color='red')
        # color by map_pval_less005

# plt.bar(np.arange(len(CGs_D6_CRoff_mc07_fracs)), CGs_D6_CRoff_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 CRISPRoff', color='deepskyblue')
# plt.bar(np.arange(len(CGs_D6_unedited_mc07_fracs)), CGs_D6_unedited_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 Unedited', color='navy')

# plt.bar(np.arange(len(CGs_D35_unedited_mc07_fracs)), CGs_D35_unedited_mc07_fracs, snap=False, alpha=0.5, label='D35 mC > 0.7 Unedited', color='red')
# mediumblue.'turquoise', 'aquamarine', 'deepskyblue', 'dodgerblue', 'cornflowerblue', 'royalblue', 'blue', 'mediumblue', 'darkblue', 'navy', 'midnightblue'

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Normalized LML Score') 
plt.title('Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
# Show the plot
plt.show()

In [ ]:
from matplotlib.patches import Patch

# color bars by boolean column 'map_pval_less005'
colors = dmr_df['map_pval_less005'].map({True: 'red', False: 'blue'}).tolist()

plt.figure(figsize=(16, 5))
x = np.arange(len(dmr_df))
y = dmr_df['score_norm']
plt.bar(x, y, color=colors, alpha=0.8, snap=False)
plt.xlabel('CpG units')
plt.ylabel('Normalized LML Score')
plt.title('Normalized LML Score per CpG \n' + experiment_codition)

plt.legend(handles=[Patch(color='red', label='map_pval_less005 = True'),
                    Patch(color='lightgray', label='map_pval_less005 = False')])
plt.show()

In [ ]:
dmr_df.columns

In [ ]:
plt.figure(figsize=(16, 5))

plt.bar(np.arange(len(dmr_df['effect_size'])), dmr_df['effect_size'], 
        snap=False, alpha=0.5, label='effect_size', color='green') #, color=dmr_df['map_pval_less005'])

plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label='score_norm', color='red') #, color=dmr_df['map_pval_less005'])

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Score') 
plt.title('Effect size & Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
plt.show()

In [ ]:
# dmr_df = df_dmr_K562_ZFPoffHigh_vs_Unedit_filtered_mC07 
# experiment_codition="K562_ZFPoffHigh_vs_Unedit_filtered_mC07"
# out_dir=dmr_dir_path
# pvalue_thresh=0.005

plt.figure(figsize=(16, 5))

plt.bar(np.arange(len(dmr_df['score_norm'])), dmr_df['score_norm'], 
        snap=False, alpha=0.5, label=experiment_codition) # , color=dmr_df['map_pval_less005'])
        # color by map_pval_less005

# plt.bar(np.arange(len(CGs_D6_CRoff_mc07_fracs)), CGs_D6_CRoff_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 CRISPRoff', color='deepskyblue')
# plt.bar(np.arange(len(CGs_D6_unedited_mc07_fracs)), CGs_D6_unedited_mc07_fracs, snap=False, alpha=0.5, label='D6 mC > 0.7 Unedited', color='navy')

# plt.bar(np.arange(len(CGs_D35_unedited_mc07_fracs)), CGs_D35_unedited_mc07_fracs, snap=False, alpha=0.5, label='D35 mC > 0.7 Unedited', color='red')
# mediumblue.'turquoise', 'aquamarine', 'deepskyblue', 'dodgerblue', 'cornflowerblue', 'royalblue', 'blue', 'mediumblue', 'darkblue', 'navy', 'midnightblue'

# Add labels and title
plt.xlabel('CpG units')
plt.ylabel('Normalized LML Score') 
plt.title('Normalized LML Score per CpG \n' + experiment_codition)
plt.legend()
# Show the plot
plt.show()

# Save dmr table

In [ ]:
date_today = '20251121'

In [ ]:
out_dir


In [ ]:
dmr_df.to_csv(os.path.join(out_dir, f'{date_today}_dmr_{experiment_codition}.csv')) # index=False)

In [ ]:
os.path.join(out_dir, f'{date_today}_dmr_{experiment_codition}.csv')

# Todo:
- plot the raw pileups together too - next too each other (have the separat individual pileup values plots above from the funcs on top)

-- chech how to look at VARIANCE per CG unit position

# TODO: check
- are there really NO diffs between the reads selected with the mC > 70 and mC > 99.5% filtering ??

could be as the mC calles are automatically selected

and in the CRoff the auto threshold 
* in mC > 70 was to 0.79
> Using filter threshold 0.7910156 for C.
* in mC > 99.5 was to  0.79
> Using filter threshold 0.7910156 for C.


and in the Unediter the auto threshold 
* in mC > 70 was to 0.8496
> Using filter threshold 0.8496094 for C.
* in mC > 99.5 was to 0.849
> Using filter threshold 0.8496094 for C.


SAME per condition AUTOMATIC modkit filtering threshold per mC run were set

(NOT 0.995 and not 0.7)